In [10]:
# import pandas as pd
# from sklearn.preprocessing import PolynomialFeatures, StandardScaler
# from sklearn.feature_selection import SelectKBest, f_classif
# from sklearn.decomposition import PCA
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
# from sklearn.metrics import accuracy_score, roc_auc_score

# # Step 1: Load train and test datasets
# train_df = pd.read_csv('train.csv')  # Adjust path if necessary
# test_df = pd.read_csv('test.csv')  # Adjust path if necessary

# # Step 2: Select numerical columns from the train dataset and target variable
# X_train = train_df.select_dtypes(include=['number']).drop('Exited', axis=1)
# y_train = train_df['Exited']

# # Step 3: Select numerical columns from the test dataset
# X_test = test_df.select_dtypes(include=['number'])  # Assumes no 'Exited' column in the test set

# # Step 4: Feature Selection (Select top 10 features based on the train data)
# selector = SelectKBest(f_classif, k=10)
# X_train_selected = selector.fit_transform(X_train, y_train)
# X_test_selected = selector.transform(X_test)

# # Step 5: Add Polynomial Features (degree=2) to both train and test sets
# poly = PolynomialFeatures(degree=2, include_bias=False)
# X_train_poly = poly.fit_transform(X_train_selected)
# X_test_poly = poly.transform(X_test_selected)

# # Step 6: Standardize both the train and test data
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train_poly)
# X_test_scaled = scaler.transform(X_test_poly)

# # Step 7: Dimensionality Reduction using PCA
# n_components = min(X_train_scaled.shape[0], X_train_scaled.shape[1], 65)  # Use 65 as an upper limit

# pca = PCA(n_components=n_components)
# X_train_reduced = pca.fit_transform(X_train_scaled)
# X_test_reduced = pca.transform(X_test_scaled)

# # Step 8: Hyperparameter Tuning using GridSearchCV
# param_grid = {
#     'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
#     'penalty': ['l2'],  # L2 penalty (Ridge-like)
#     'solver': ['lbfgs'],  # Solver for optimization
#     'max_iter': [500, 1000]
# }

# # Step 9: Cross-validation and model training
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# logistic_model = LogisticRegression()

# grid_search = GridSearchCV(logistic_model, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
# grid_search.fit(X_train_reduced, y_train)

# # Best Logistic Regression model after cross-validation
# best_model = grid_search.best_estimator_

# # Step 10: Evaluate model on the training set
# y_train_pred = best_model.predict(X_train_reduced)
# y_train_pred_proba = best_model.predict_proba(X_train_reduced)[:, 1]

# accuracy_train = accuracy_score(y_train, y_train_pred)
# roc_auc_train = roc_auc_score(y_train, y_train_pred_proba)

# print(f'Best Hyperparameters: {grid_search.best_params_}')
# print(f'Training Accuracy: {accuracy_train:.4f}')
# print(f'Training ROC-AUC: {roc_auc_train:.4f}')

# # Step 11: Make predictions on the test set
# test_pred = best_model.predict_proba(X_test_reduced)[:, 1]  # Probability of exiting (class 1)

# # Step 12: Save the predictions in the test DataFrame
# test_df['Exited'] = test_pred

# # Ensure correct column names are used for output (replace 'CustomerId' with the actual identifier)
# # If your test file has a different identifier column, adjust 'CustomerId' to the correct name
# if 'CustomerId' in test_df.columns:
#     test_df[['id', 'Exited']].to_csv('submission3.csv', index=False)
# else:
#     print("Error: 'id' column not found in test data. Adjust the identifier column name.")
#     test_df.to_csv('submission3.csv', index=False)  # Save the entire test DataFrame if 'CustomerId' not found

# print("Submission file saved as 'submission3.csv'.")


Best Hyperparameters: {'C': 100, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
Training Accuracy: 0.8509
Training ROC-AUC: 0.8653
Submission file saved as 'submission3.csv'.


In [14]:
# import pandas as pd
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.pipeline import Pipeline  # Add this line to import Pipeline
# from sklearn.metrics import roc_auc_score, accuracy_score
# import numpy as np

# # Step 1: Load train and test datasets
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

# # Store the 'id' column for the final submission
# test_df_id = test_df[['id']].copy()

# # Step 2: Drop unnecessary columns and ensure necessary transformations
# train_df = train_df.drop(columns=['id', 'Surname'])
# test_df = test_df.drop(columns=['id', 'Surname'])

# # Step 3: Encode categorical variables (Geography, Gender)
# from sklearn.preprocessing import LabelEncoder
# lbl = LabelEncoder()
# train_df['Geography'] = lbl.fit_transform(train_df['Geography'])
# test_df['Geography'] = lbl.transform(test_df['Geography'])
# train_df['Gender'] = lbl.fit_transform(train_df['Gender'])
# test_df['Gender'] = lbl.transform(test_df['Gender'])

# # Step 4: Feature engineering - WoE transformations, scaling, interaction terms
# train_df['Balance'] = np.round(train_df['Balance'])
# train_df['EstimatedSalary'] = np.round(train_df['EstimatedSalary'])
# test_df['Balance'] = np.round(test_df['Balance'])
# test_df['EstimatedSalary'] = np.round(test_df['EstimatedSalary'])

# # Create interaction terms in both train and test sets
# train_df['Age_x_EstimatedSalary'] = train_df['Age'] * train_df['EstimatedSalary']
# test_df['Age_x_EstimatedSalary'] = test_df['Age'] * test_df['EstimatedSalary']

# # Step 5: Define features and target
# X = train_df.drop(columns=['Exited'])
# y = train_df['Exited']

# # Make sure the test dataset has the same features as X
# test_X = test_df[X.columns]

# # Step 6: Specify categorical and numerical features for preprocessing
# categorical_features = ['Geography', 'Gender']
# numerical_features = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# # Step 7: Create the preprocessor pipeline for numeric and categorical features
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), numerical_features),
#         ('cat', OneHotEncoder(), categorical_features)
#     ]
# )

# # Step 8: Create a pipeline with polynomial features and Logistic Regression
# model = Pipeline([
#     ('preprocessor', preprocessor),
#     ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Applies polynomial features to numerical columns
#     ('classifier', LogisticRegression(max_iter=1000))
# ])

# # Step 9: Split the data into training and validation sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

# # Step 10: Hyperparameter Tuning using GridSearchCV
# param_grid = {
#     'classifier__C': [0.001, 0.01, 0.1, 1, 10],  # Regularization strength
#     'classifier__solver': ['lbfgs'],  # Logistic regression solver
#     'classifier__max_iter': [0, 10000, 50000]
# }

# cv = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
# cv.fit(X_train, y_train)

# # Best estimator
# best_model = cv.best_estimator_

# # Step 11: Make predictions on the validation set
# y_pred = best_model.predict(X_test)
# y_score = best_model.predict_proba(X_test)[:, 1]

# # Evaluate the model
# roc_auc = roc_auc_score(y_test, y_score)
# accuracy = accuracy_score(y_test, y_pred)

# print(f'Validation ROC AUC: {roc_auc}')
# print(f'Validation Accuracy: {accuracy}')

# # Step 12: Make predictions on the test set
# test_pred_proba = best_model.predict_proba(test_X)[:, 1]

# # Round the predicted probabilities to 0.01
# test_pred = np.round(test_pred_proba, 3)

# # Step 13: Save the test predictions in the test DataFrame
# test_df_id['Exited'] = test_pred

# # Step 14: Save final submission with 'id' column
# test_df_id[['id', 'Exited']].to_csv('submission_final.csv', index=False)

# print("Submission file saved as 'submission_final.csv'.")


Validation ROC AUC: 0.8817469989677418
Validation Accuracy: 0.8632257353751239
Submission file saved as 'submission_final.csv'.


In [16]:
! pip install statsmodels


  Obtaining dependency information for statsmodels from https://files.pythonhosted.org/packages/4b/e4/f9e96896278308e17dfd4f60a84826c48117674c980234ee38f59ab28a12/statsmodels-0.14.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for patsy>=0.5.6 from https://files.pythonhosted.org/packages/43/f3/1d311a09c34f14f5973bb0bb0dc3a6e007e1eda90b5492d082689936ca51/patsy-0.5.6-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ----------------------


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import RidgeClassifier, Lasso
from sklearn.ensemble import RandomForestClassifier
from phik import phik_matrix
from sklearn.feature_selection import mutual_info_classif
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import KBinsDiscretizer
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Load train and test datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Step 2: Store 'id' column for the final submission
test_df_id = test_df[['id']].copy()

# Step 3: Drop unnecessary columns and ensure necessary transformations
train_df = train_df.drop(columns=['id', 'Surname'])
test_df = test_df.drop(columns=['id', 'Surname'])

# Step 4: Encode categorical variables (Geography, Gender)
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
train_df['Geography'] = lbl.fit_transform(train_df['Geography'])
test_df['Geography'] = lbl.transform(test_df['Geography'])
train_df['Gender'] = lbl.fit_transform(train_df['Gender'])
test_df['Gender'] = lbl.transform(test_df['Gender'])

# Step 5: Feature engineering - WoE, scaling, interaction terms, and binning
train_df['Balance'] = np.round(train_df['Balance'])
train_df['EstimatedSalary'] = np.round(train_df['EstimatedSalary'])
test_df['Balance'] = np.round(test_df['Balance'])
test_df['EstimatedSalary'] = np.round(test_df['EstimatedSalary'])

# Interaction terms
train_df['Age_x_EstimatedSalary'] = train_df['Age'] * train_df['EstimatedSalary']
test_df['Age_x_EstimatedSalary'] = test_df['Age'] * test_df['EstimatedSalary']

# Binning Features
bin_features = ['EstimatedSalary', 'Balance', 'CreditScore']
for feature in bin_features:
    est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
    train_df[feature+'_binned'] = est.fit_transform(train_df[[feature]])
    test_df[feature+'_binned'] = est.transform(test_df[[feature]])

# Step 6: Define features and target
X = train_df.drop(columns=['Exited'])
y = train_df['Exited']

# Make sure the test dataset has the same features as X
test_X = test_df[X.columns]

# Step 7: Specify categorical and numerical features
categorical_features = ['Geography', 'Gender']
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Step 8: Create the preprocessor pipeline for numeric and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Step 9: Create a stacking classifier with multiple base models
base_models = [
    ('ridge', RidgeClassifier()),
    ('lasso', Lasso(max_iter=1000)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
]

stacking_clf = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(max_iter=1000))

# Step 10: Create the pipeline with polynomial features and StackingClassifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('classifier', stacking_clf)
])

# Step 11: Hyperparameter tuning using GridSearchCV
param_grid = {
    'classifier__final_estimator__C': [0.1, 1, 10, 100],  # Logistic Regression regularization strength
    'classifier__final_estimator__solver': ['lbfgs'],
    'classifier__final_estimator__max_iter': [500]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X, y)

# Best model after hyperparameter tuning
best_model = grid_search.best_estimator_

# Step 12: Make predictions and evaluate model performance
y_pred = best_model.predict(X)
y_score = best_model.predict_proba(X)[:, 1]

roc_auc_train = roc_auc_score(y, y_score)
accuracy_train = accuracy_score(y, y_pred)

print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Training Accuracy: {accuracy_train:.4f}")
print(f"Training ROC-AUC: {roc_auc_train:.4f}")

# Step 13: Make predictions on the test set
test_pred_proba = best_model.predict_proba(test_X)[:, 1]

# Round the predicted probabilities to 0.001
test_pred = np.round(test_pred_proba, 3)

# Step 14: Save the test predictions in the test DataFrame
test_df_id['Exited'] = test_pred
test_df_id.to_csv('submission_final11.csv', index=False)

print("Submission file saved as 'submission_final11.csv'.")


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
